In [1]:
from random import *
import csv

In [2]:
DNA_codon_table = [
        ("ATG","M/Start"), ("ATA","I"), ("ATC","I"), ("ATT","I"), ("ACG","T"), ("ACA","T"),
        ("ACC","T"), ("ACT", "T"), ("AAG","K"), ("AAA","K"), ("AAC","N"), ("AAT","N"),
        ("AGG","R"), ("AGA","R"), ("AGC","S"), ("AGT","S"), 
        ("GTG","V"), ("GTA","V"), ("GTC","V"), ("GTT","V"), ("GCG","A"), ("GCA","A"), ("GCC","A"), ("GCT","A"), 
        ("GAG","E"), ("GAA","E"), ("GAC","D"), ("GAT","D"), ("GGG","G"), ("GGA","G"), ("GGC","G"), ("GGT","G"), 
        ("CTG","L"), ("CTA","L"), ("CTC","L"), ("CTT","L"), ("CCG","P"), ("CCA","P"), ("CCC","P"), ("CCT","P"),
        ("CAG","Q"), ("CAA","Q"), ("CAC","H"), ("CAT","H"), ("CGG","R"), ("CGA","R"), ("CGC","R"), ("CGT","R"),
        ("TTG","L"), ("TTA","L"), ("TTC","F"), ("TTT","F"), ("TCG","S"), ("TCA","S"), ("TCC","S"), ("TCT","S"),
        ("TAC","Y"), ("TAT","Y"), ("TGG","W"), ("TGC","C"), ("TGT","C"), ("TAG","Amber/Stop"),
        ("TAA","Ochre/Stop"),("TGA","Opal/Stop")
]

Nucl = ["A", "T", "G", "C"]

stop = ["TAG", "TAA", "TGA"]

In [3]:
#rules:
# -1 = default
# 0 - 4 = exclude Nucleotide
def create_protein(seq_length, codon_excl, rule=-1, rand_pos_pick=-1, AminExcl=0): #AminExcl -# amino acids to exclude
    protein = []
    tmp = []
    Excl = [] # " "
    #if AminExcl == 1:
    #    Excl = DNA_codon_table[randint(1, 61)][1]
    tmp_table = DNA_codon_table[1:61]
    limit_table = []
    #print(tmp_table)

    if AminExcl > 0: # > 1
        #Excl = []
        for i in range(AminExcl):
            #print(len(tmp_table))
            tmp = tmp_table[randint(0, len(tmp_table)-1)]
            #while(tmp[1] in Excl):
                #print(tmp)
                #tmp_table.remove(tmp)
                #tmp = tmp_table[randint(0, len(tmp_table)-1)]
            Excl.append(tmp[1])
            tmp_table.remove(tmp)
            
    for i in range(1, 60):
        if(DNA_codon_table[i][1] not in codon_excl):
            limit_table.append(DNA_codon_table[i])
            
    #print("Amino Acid exclusion: ", Excl)
    triplet = limit_table[randint(0, len(limit_table)-1)]
    for j in range(0, seq_length):  #generating one specific protein sequence
        if(rule == -1):
            #protein.append(aminos[randint(0, len(aminos)-1)])  #DNA sequences
            if(rand_pos_pick > -1):
                if(rand_pos_pick == j):
                    protein.append(triplet[0])
                else:
                    protein.append(DNA_codon_table[randint(1, 60)][0])
            else:
                protein.append(DNA_codon_table[randint(1, 60)][0])
            
        else:
            tmp = DNA_codon_table[randint(1, 60)][0]
            #aminos[randint(0, len(aminos)-1)]
            while(Nucl[rule % 4] in tmp):
                tmp = DNA_codon_table[randint(1, 60)][0]
                #aminos[randint(0, len(aminos)-1)]
            protein.append(tmp)
    return protein, Excl, triplet[1]

In [4]:
def num_comb(val, codon_dict):
    pos_comb = 0
    for key, value in codon_dict.items():
        if value == val:
            pos_comb += 1
    return pos_comb

In [5]:
def find_comb(val, codon_dict, rule=-1):
    combs = []
    for key, value in codon_dict.items():
        if value == val:
            if(rule != -1):
                if(not Nucl[rule % 4] in key):
                    combs.append(key)
            else:
                combs.append(key)
    return combs

In [6]:
def create_sequences(protein, num_seq, rule=-1, rand_pos_pick=-1, rand_change=0):
    '''
        1. Protein sequence
        2. used_key lists for each amino acid
                           -OR-
           lists of possible key values for each key
    '''
    sequence = []
    tmp = []
    amino_list = []
    protein_len = len(protein)
    for i in range(0, protein_len):
        amino_list.append(False)
    amino_list[0] = True
    codon_dict = dict(DNA_codon_table)
    #print(codon_dict)
    seq_count = 0
    comb_dict = {}
    #comb_list = []
    for i in range(0, len(protein)):
        #print(codon_dict[protein[i]])
        combs = find_comb(codon_dict[protein[i]], codon_dict, rule)
        #print(type(comb_dict))
        comb_dict[i] = combs
        #comb_list.append(combs)
    #print(comb_dict)
    index = 0
    comb_tracker = [0 for i in range(0,len(protein))]
    #print(comb_tracker)

    while seq_count < num_seq:
        for i in range(0, protein_len):
            select = DNA_codon_table[randint(1, 60)][0]
            if i == rand_pos_pick:
                select = choice(comb_dict[i])
            tmp.append(select)
        sequence.append(list(tmp))
        tmp = []
        seq_count += 1
        
    #print(sequence)
    #print("amount of sequences: ", len(sequence))
        
    if rand_change > 0:
        rand_samples = randint(1, num_seq-1)  #alter rand_samples amount of sequences in protein class
        #print("rand_samples: ", rand_samples)
        for i in range(rand_samples):
            which_samp = randint(0, num_seq-1) #which seq to modify
            #print("Which samp: ", which_samp)
            for j in range(rand_change):   #which amino acids to modify
                loc = randint(0, protein_len-1)
                #print("location: ", loc)
                sequence[which_samp][loc] = DNA_codon_table[randint(1, 60)][0]
            
            
    return sequence
    

In [7]:
def create_set(num_aminos, seq_length, num_classes, seq_per_class):
    s = ""
    if num_aminos < 3:
        print("Need more aminos")
        return
    rand_aminos = []
    classes = []
    seq = []
    Excl = []
    codonExcl = []
    #for i in range(1, num_aminos-3): # -2 because of start and stop codons
    #    rand_aminos.append(DNA_codon_table[randint(1, 62)][0])  #looking at DNA

    for i in range(0, num_classes):  #generating diffferent proteins
        protein, tmp, CExcl = create_protein(seq_length, codonExcl, -1, rand_pos_pick=(seq_length-1))
        Excl.append(tmp)
        codonExcl.append(CExcl)
        print("protein ", protein)
        seq_list = create_sequences(protein, seq_per_class, rand_pos_pick=(seq_length-1))
        for j in seq_list:
        #    j.insert(0, "ATG")
        #    j.append(choice(stop))
            seq.append(''.join(j))
            classes.append(i)
            print(j)
            print()
        #print(type(seq_new))
        print("Finished class: ", i+1)
    #print(seq)
    print(codonExcl)
    return classes, seq, Excl

In [8]:
num_aminos = 64
seq_length = 50
num_classes = 19
seq_per_class = 500
keys, val, Excl = create_set(num_aminos, seq_length, num_classes, seq_per_class)
print(len(val))
#print(keys)
#print(len(val))
#list of tuples (x,y) where x is a string and y is index
fout = str(num_aminos) + '_' + str(seq_length) + '_' + str(num_classes) + '_' + str(seq_per_class) + '_rand_last'
print("data"+fout)
#for i in range(len(Excl)):
    #print("Amino Acid exclusion ", i+1, ": ", Excl[i])
with open('../data/data%s.csv' % fout, 'w') as f:
#with open('data/64_30_30_10.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['class', 'sequence'])
    for i in range(0, len(keys)):
        #print("i: ", i)
        writer.writerow([keys[i], val[i]])

protein  ['GGA', 'GCG', 'ACA', 'AGG', 'GAC', 'GCC', 'AAG', 'CCA', 'CCG', 'TGC', 'CTT', 'CGG', 'GGT', 'ATT', 'CGT', 'GTT', 'TCA', 'TAT', 'ACC', 'CCC', 'GAC', 'AAA', 'CGA', 'CGG', 'GTG', 'ATT', 'TCG', 'GCA', 'CGG', 'ATT', 'ATC', 'CCT', 'TCT', 'AGA', 'CGG', 'CTG', 'GAG', 'CAC', 'TGT', 'TCC', 'AGT', 'TTC', 'TTA', 'CAC', 'CCA', 'AAG', 'ACA', 'AGA', 'CAC', 'CAA']
['GTG', 'CCC', 'TAC', 'TTA', 'GTG', 'CGA', 'CCT', 'TCT', 'TGG', 'CGT', 'ACG', 'GTA', 'GAG', 'CAA', 'GAC', 'AAT', 'GTA', 'GTG', 'CGG', 'CAC', 'ATC', 'CCA', 'TCA', 'GCC', 'TAT', 'CTT', 'GGT', 'ACA', 'TTG', 'AGC', 'TCG', 'CAA', 'ATA', 'TCG', 'GGG', 'GCC', 'CCT', 'AAT', 'TCA', 'CCT', 'TGG', 'CCG', 'GAA', 'CCG', 'CAT', 'CCA', 'TCA', 'CAC', 'CGT', 'CAA']

['GCG', 'TAT', 'GAG', 'TGG', 'TCA', 'CAT', 'CTA', 'CAT', 'TAC', 'CGT', 'ACT', 'ACC', 'TTT', 'CCA', 'TAC', 'GTA', 'ATA', 'AAT', 'CTA', 'CAC', 'TTC', 'TCG', 'TCT', 'CGC', 'CGA', 'CTC', 'AGG', 'TTA', 'GGG', 'TTC', 'TAT', 'CCG', 'TCA', 'TCG', 'TTT', 'TTA', 'GCA', 'AGT', 'TCG', 'CAT', 'TGC', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

